# IRC2 TOKEN


In [1]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os

In [2]:
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))

In [3]:
NID = 3


In [4]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# CREATE WALLET

In [6]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

hx2352046d9630738d0f612db4093b3414308560e9
hx6bd274789397013eb0a09ff22f924462eebdbfb7


In [7]:
print(deployer_wallet.get_private_key())
print(caller_wallet.get_private_key())

87441f53900dd7c2f6f7a89344e5d8de625e4726b05a0fa8a45e14bcf500355b
dfe53b7c3a7ebe60ad2a11775765ae23c9d926e7301ab54c58b87b54e615d1d2


In [8]:
deployer_priv_key = "87441f53900dd7c2f6f7a89344e5d8de625e4726b05a0fa8a45e14bcf500355b"
caller_priv_key = "dfe53b7c3a7ebe60ad2a11775765ae23c9d926e7301ab54c58b87b54e615d1d2"

# The private key needs to be converted to bytes before we can load the address.
deployer_wallet_priv = bytes.fromhex(deployer_priv_key)
caller_wallet_priv = bytes.fromhex(caller_priv_key)

In [9]:
deployer_wallet = KeyWallet.load(deployer_wallet_priv) 
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.load(caller_wallet_priv) 
print(caller_wallet.get_address())

hx2352046d9630738d0f612db4093b3414308560e9
hx6bd274789397013eb0a09ff22f924462eebdbfb7


# DEPLOY

In [19]:
DEPLOY_PARAMS =  {
            "_name": "TestToken",
            "_symbol": "TK",
            "_initialSupply": 1000,
            "_decimals": 18
        }


deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx0000000000000000000000000000000000000000")\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('sampletoken'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)
SCORE_ADDRESS = tx_result['scoreAddress']
SCORE_ADDRESS

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


'cx9b48815344bab7b03e026b8df263b2bc3ae6d5de'

# Check the name, symbol, decimals and total supply

In [14]:
external_methods = ["name", "symbol", "decimals", "totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(deployer_wallet.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

TestToken
TK
0x12
0x3635c9adc5dea00000


# Check the total number of tokens/ balance of caller and deployer address

In [15]:
external_methods = [deployer_wallet.get_address(), caller_wallet.get_address()]
for address in external_methods:
    call = CallBuilder().from_(deployer_wallet.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'_owner': address})\
                    .build()
    print(icon_service.call(call))

0x3635c9adc5dea00000
0x0


# Transfering Token

In [16]:
params={
    "_to": caller_wallet.get_address(),
    "_value": 5,
}

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()


estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xac1f79ad4024c8602c6b55e15b1b9d1ff3a0a6affe3530592c771048a594c49a',
 'blockHeight': 13240651,
 'blockHash': '0x13e8336001060c73ec2aaac25380ab95c47fb5ebdb52ea4506ed6e9fb996e94e',
 'txIndex': 1,
 'to': 'cx9b48815344bab7b03e026b8df263b2bc3ae6d5de',
 'stepUsed': 153840,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 153840,
 'eventLogs': [{'scoreAddress': 'cx9b48815344bab7b03e026b8df263b2bc3ae6d5de',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx2352046d9630738d0f612db4093b3414308560e9',
    'hx6bd274789397013eb0a09ff22f924462eebdbfb7',
    '0x5'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

# gvkvuv

# Updating Contract

In [25]:
update_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('sampletoken'))\
    .build()

estimate_step = icon_service.estimate_step(update_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(update_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x061925e9cf9c7857620fea6cdc9fbb3391d714fafb72a6693812303d00b883b3',
 'blockHeight': 13241236,
 'blockHash': '0x5889810ab59f531e7568747615a5b439fff71c1b3d29052c9808e5286594ca0e',
 'txIndex': 1,
 'to': 'cx9b48815344bab7b03e026b8df263b2bc3ae6d5de',
 'scoreAddress': 'cx9b48815344bab7b03e026b8df263b2bc3ae6d5de',
 'stepUsed': 2067631200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2067631200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Testing mint Method


In [27]:
params={
    "_amount": 5,
}

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("mint")\
    .params(params)\
    .build()


estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x6efd98a36f732fa9ef454b803c5c026d24791d0d8bc2c04dde5562986f572006',
 'blockHeight': 13244275,
 'blockHash': '0x0107a8517c0b93c793dbde44860f50503dda72017f5ed5220b7d793b4429c07f',
 'txIndex': 1,
 'to': 'cx9b48815344bab7b03e026b8df263b2bc3ae6d5de',
 'stepUsed': 139140,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 139140,
 'eventLogs': [{'scoreAddress': 'cx9b48815344bab7b03e026b8df263b2bc3ae6d5de',
   'indexed': ['Mint(Address,int)',
    'hx2352046d9630738d0f612db4093b3414308560e9'],
   'data': ['0x5']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

# Testing mintTo Method

In [2]:
params={
    "_account": caller_wallet.get_address(),
    "_amount": 5,
}

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("mintTo")\
    .params(params)\
    .build()


estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result

NameError: name 'caller_wallet' is not defined